# IEX Cloud API Access samples

Based on the course [Algorithmic Trading Using Python - Full Course](https://www.youtube.com/watch?v=xfzGZB4HhEE) by Nick McCullum from Youtube channel [freeCodeCamp.org](https://www.youtube.com/channel/UC8butISFwT-Wl7EV0hUK0BQ).

* Main site: https://iexcloud.io/
* Documentation: https://iexcloud.io/docs/api/

In [1]:
import numpy as np
import pandas as pd
import requests

## Acquiring an API Token

Create account in https://iexcloud.io/ Tokens are available in https://iexcloud.io/console/tokens

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get published to your local Git repository.

In [2]:
from secrets import IEX_CLOUD_API_TOKEN

Set a list of some stocks

In [3]:
stocks = ['A', 'AAL','AAP','AAPL','ABBV','ABC','ABMD','ABT', 'ACN']

## Price and Market Capitalization API Call

Note than in [Algorithmic Trading Using Python - Full Course](https://www.youtube.com/watch?v=xfzGZB4HhEE) the `https://sandbox.iexapis.com/` is used. It provides simulated test data.

However for real market data the `https://cloud.iexapis.com/` should be used. Note that some of the functions may require payment. Check details in your account.

In [4]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url)

Check the status of the request. It should be 200

In [5]:
data.status_code

200

Convert the result to JSON:

In [6]:
data.json()

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'NASDAQ/NGS (GLOBAL SELECT MARKET)',
 'calculationPrice': 'close',
 'open': 137.35,
 'openTime': 1612535400523,
 'openSource': 'official',
 'close': 136.76,
 'closeTime': 1612560560507,
 'closeSource': 'official',
 'high': 137.42,
 'highTime': 1612558798721,
 'highSource': '15 minute delayed price',
 'low': 135.86,
 'lowTime': 1612538572358,
 'lowSource': '15 minute delayed price',
 'latestPrice': 136.76,
 'latestSource': 'Close',
 'latestTime': 'February 5, 2021',
 'latestUpdate': 1612560560507,
 'latestVolume': 75693830,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': 136.77,
 'delayedPriceTime': 1612558798721,
 'oddLotDelayedPrice': 136.76,
 'oddLotDelayedPriceTime': 1612558798681,
 'extendedPrice': 136.66,
 'extendedChange': -0.1,
 'extendedChangePercent': -0.00073,
 'extendedPriceTime': 1612573198054,
 'previousClose': 137.185,
 'previousVolume': 84183061,
 'change': 

Print price and data capitalization

In [7]:
price = data.json()['latestPrice']
market_cap = data.json()['marketCap']
price, market_cap / 1000000000000

(136.76, 2.29594000896)

In [8]:
data.json()['peRatio']

37.16

## Looping Through The Tickers in List of Stocks

Using the same logic outlined above, the data can be pulled for stocks in the list.

**Note** _This is not recommended as it is slow to execute and is priced per **each** request. Use the batch approach to be described further._

In [9]:
dataframe_columns = ['Ticker', 'Stock Price', 'Market Capitalization']
dataframe = pd.DataFrame(columns=dataframe_columns)

for stock in stocks:
    api_url = f'https://cloud.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    dataframe = dataframe.append(
        pd.Series(
            [
                stock,
                data['latestPrice'],
                data['marketCap']
            ],
            index = dataframe_columns
        ),
        ignore_index=True
    )

In [10]:
dataframe

,Ticker,Stock Price,Market Capitalization
0,A,123.18,37797724613
1,AAL,17.19,10399929544
2,AAP,157.28,10672161894
3,AAPL,136.76,2295940008960
4,ABBV,108.73,191959979648
5,ABC,109.69,22404154748
6,ABMD,327.85,14828972203
7,ABT,124.03,219826006891
8,ACN,253.54,167624200860


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of the code.

This is because HTTP requests are typically one of the slowest components of a scripts.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement in this section, we'll split our list of stocks of 100 and then make a batch API call for each group.

In [11]:
symbol_strings = ','.join(stocks)
symbol_strings

'A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN'

In [12]:
batch_dataframe = pd.DataFrame(columns=dataframe_columns)

batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_strings}&types=quote&token={IEX_CLOUD_API_TOKEN}'

data = requests.get(batch_api_call_url).json()
for symbol in symbol_strings.split(','):
    batch_dataframe = batch_dataframe.append(
        pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap']
            ],
            index = dataframe_columns
        ),
        ignore_index=True
    )

In [13]:
batch_dataframe

,Ticker,Stock Price,Market Capitalization
0,A,123.18,37797724613
1,AAL,17.19,10399929544
2,AAP,157.28,10672161894
3,AAPL,136.76,2295940008960
4,ABBV,108.73,191959979648
5,ABC,109.69,22404154748
6,ABMD,327.85,14828972203
7,ABT,124.03,219826006891
8,ACN,253.54,167624200860


## Read Key Stats

In [14]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url)
data.status_code

200

In [15]:
data.json()

{'companyName': 'Apple Inc',
 'marketcap': 2295940008960,
 'week52high': 142.95,
 'week52low': 55.66,
 'week52change': 0.7225895271695797,
 'sharesOutstanding': 16788096000,
 'float': 0,
 'avg10Volume': 108492565,
 'avg30Volume': 108727355,
 'day200MovingAvg': 119.64,
 'day50MovingAvg': 133.12,
 'employees': 0,
 'ttmEPS': 3.68,
 'ttmDividendRate': 1.0090288065748436,
 'dividendYield': 0.007378098907391369,
 'nextDividendDate': '',
 'exDividendDate': '2021-02-05',
 'nextEarningsDate': '2021-01-27',
 'peRatio': 35.913342858751754,
 'beta': 1.14690182912981,
 'maxChangePercent': 51.40047511398904,
 'year5ChangePercent': 5.255546102405064,
 'year2ChangePercent': 2.2201857328536176,
 'year1ChangePercent': 0.7190514797845529,
 'ytdChangePercent': 0.03221326570660876,
 'month6ChangePercent': 0.2488186172228486,
 'month3ChangePercent': 0.15265614082700285,
 'month1ChangePercent': 0.045449854565051906,
 'day30ChangePercent': 0.08186701958926057,
 'day5ChangePercent': 0.0379233639767127}

In [16]:
data.json()['year1ChangePercent']

0.7190514797845529

In [17]:
data.json()['month6ChangePercent']

0.2488186172228486

In [18]:
data.json()['month3ChangePercent']

0.15265614082700285

In [19]:
data.json()['month1ChangePercent']

0.045449854565051906

## Read Advanced Key Stats

In [20]:
symbol = 'AAPL'
batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/advanced-stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url)

In [21]:
data.status_code

402

402 code means that payment is required.